# Analysis of the XCSP'19 competition (creation and manipulations)

## `Metrics-wallet` is imported

In [1]:
from metrics.wallet import *

## Analysis is created with consistency checkings

In [2]:
inconsistent_returns = {
    'ERR WRONGCERT', 'ERR UNSAT'
}

successful_returns = {'SAT', 'UNSAT'}

is_consistent_by_xp = (lambda x: not x['Checked answer'] in inconsistent_returns)
is_consistent_by_input = (lambda df: len(set(df['Checked answer'].unique()) & successful_returns) < 2)
is_success = (lambda x: x['Checked answer'] in successful_returns)

analysis = Analysis(input_file='config/metrics_scalpel.yml')

analysis.check_success(is_success)
analysis.check_input_consistency(is_consistent_by_input)
analysis.check_xp_consistency(is_consistent_by_xp)

2100 experiments are missing and have been added as unsuccessful.
31 experiments are inconsistent and are declared as unsuccessful.


## Parallel experiment-wares are removed

In this analysis, we are only considering sequential experiment-wares.

In [3]:
analysis = analysis.filter_analysis(lambda x: 'parallel' not in x['experiment_ware'])\
    .filter_analysis(lambda x: 'Many' not in x['experiment_ware'])

## Duplicated experiment-wares are removed

In [4]:
analysis = analysis.remove_experiment_wares({
    'Concrete 3.12.2', 
    'cosoco 2.0'
})

## Only CSP instances

In [5]:
analysis = analysis.filter_analysis(lambda x: 'CSP' == x['Category'])

## The error table

Only two instances are inconsistent due to their `Checked answer` parameter and are considered unsuccessful by the analysis.

In [6]:
analysis.error_table()

input          experiment_ware     cpu_time Category  \
13473  XCSP19/hcp/graph255.xml                 cosoco 2     0.045418      CSP   
13498   XCSP19/hcp/graph48.xml  choco-solver 2019-09-16  2306.850000      CSP   

      Checked answer Objective function  Wallclock time      Memory  \
13473      ERR UNSAT                           0.045142         0.0   
13498  ERR WRONGCERT                         583.697000  15530540.0   

        Solver name Solver version  timeout  success  user_success  missing  \
13473        cosoco              2   2400.0    False         False    False   
13498  choco-solver     2019-09-16   2400.0    False         False    False   

       consistent_xp  consistent_input  error  
13473          False              True   True  
13498          False              True   True

## A new column: `family`

In [7]:
import re
family_re = re.compile(r'^XCSP\d\d/(.*?)/')

analysis = analysis.add_variable(
    new_var='family', 
    function=lambda x: family_re.match(x['input']).group(1)
)

## Unnecessary variables

In [8]:
analysis = analysis.remove_variables(
    vars=['Category', 'Objective function']
)

## The VBS is added

In [9]:
analysis = analysis.add_virtual_experiment_ware(
    function=find_best_cpu_time_input, 
    xp_ware_set=None, 
    name='VBS'
)

## The analysis is exported!

In [10]:
analysis.export('analysis.csv')